# EN PROCESO

 **OBJETIVOS**
 1. Sacar nubes de palabras por personaje (a ser posible con la forma de cada personaje)
 2. Estudiar qué personaje dice más una palabra específica (jew, fuck, etc.) o frases características (you're breaking my balls, mooom...)
 3. Análisis de sentimientos por personaje, por temporada
 4. Comparar vocabulario usado con puntuación en IMDB (¿influye?)
 5. Analizar cómo varía la participación de personajes de distinta etnia o género a lo largo de la serie
 6. Ver cómo fluctúa la participación de cada personaje en cada temporada de la serie

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import difflib
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import urllib
import requests
from matplotlib import pyplot as plt
import imageio
import textblob
import nltk
import twython

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
## UTILS

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [ ]:
df = pd.read_csv("../input/southparklines/All-seasons.csv")

In [ ]:
df.head()

**Ordenamos las líneas de diálogo por la temporada y por el capítulo** 

*Antes quitamos las líneas de 'marcaje' (las que separan las temporadas)*

In [ ]:
df['Season'].unique()
df = df[df['Season']!='Season']
df['Season'] = [int(season) for season in df['Season']]
df['Episode'] = [int(episode) for episode in df['Episode']]
df = df.sort_values(by=['Season', 'Episode'], ignore_index=True)

In [ ]:
len(df)

In [ ]:
df.head()

# Comprobamos distintas hipótesis
1. ** ¿Las canciones cuentan como diálogo?**

Sabemos que el capítulo 7x09 ***Christian Rock Hard*** Cartman tiene un grupo de rock cristiano. ¿Aparecerán las canciones como diálogo?

In [ ]:
sing = df[(df['Season']==7) & (df['Episode']==9)]

sing

for i in range(len(sing[:8])):
    print(sing.iloc[i,:]['Character'] + ': ' + sing.iloc[i,:]['Line'] + '\n')

¡Parece que sí cuenta las canciones!

2. *¿Se cuentan las frases de Jimmy como diálogo? ¿Starvin' Marvin aparece como personaje? ¿Salen los balbuceos de Timmy? etc.*

Vamos a hacer una función que nos de el guion de un capítulo en particular, para poder leerlo y dar respuesta a estas preguntas.

In [ ]:
## UTILS

class color:
   UNDERLINE = '\033[4m'
   END = '\033[0m'

def df_read_episode(season, episode, max_number_of_lines=None):
    df_script = df[(df['Season']==season) & (df['Episode']==episode)]
    if max_number_of_lines is None: ## Mejorar eficiencia
        for i in range(len(df_script)):
            print(color.UNDERLINE + df_script.iloc[i,:]['Character'] + color.END + ': ' + df_script.iloc[i,:]['Line'] + '\n')
    else:
        for i in range(len(df_script[:max_number_of_lines])):
            print(color.UNDERLINE + df_script.iloc[i,:]['Character'] + color.END + ': ' + df_script.iloc[i,:]['Line'] + '\n')
            
#Capítulo: 'Starvin' Marvin'
df_read_episode(season = 1, episode = 8, max_number_of_lines=8)

# **1. Limpiamos la columna de personajes**

# 1.1. Personajes grupales

Se consideran como personajes *Boys y Kids*. Entendiendo que se refieren a los cuatro chicos protagonistas, sustituimos cada línea con ese *Character* asociado por cuatro líneas, una para cada uno de los chicos.

In [ ]:
for i in range(len(df)):
    line = df.iloc[i,:]
    if line['Character'] == 'Boys' or line['Character'] == 'Kids':
        for character in ['Stan','Kyle','Cartman','Kenny']:
            line['Character'] = character
            df = df.append(line, ignore_index=True)
            
df = df[df['Character']!='Boys']        
df = df[df['Character']!='Kids']       
df = df.sort_values(by=['Season', 'Episode'], ignore_index=True)

#  1.2. Quitamos el carácter '\n' del final de las líneas
 
 NOTA: Hay varias líneas que tienen '...' como reducción de texto. Parece que no son muchas.. A mejorar en futuro.



In [ ]:
## OPTIMIZAR

#for i in range(len(df)):
#    line = df.iloc[i,:]
#    line_Line = line['Line']
#    last_characters = line_Line[-1:]
#    if last_characters=='\n':
#        line['Line'] = line_Line[:-1]
#        df.iloc[i,:] = line
       

# 1.3. Varios personajes hablan a la vez 
 
Si hay varios personajes que dicen lo mismo al mismo tiempo, los separa por comas, guiones o por un 'and'. Haremos una línea para cada personaje involucrado.


In [ ]:
# POR HACER

# Hay 215 filas, algunas dan problemas

df_commas = [characters for characters in df['Character'] if (',' in characters or '/' in characters)]

dict_commas_conflictive = {
    'Celine, Terrance, Bob, Phillip, and people on the field' : 'Celine, Terrance, Bob, Phillip',
    'Boys 1/2': 'Stan, Kyle, Kenny, Cartman',
    'Lola, Jenny, Red, and Wendy' : 'Lola, Jenny, Red, Wendy',
    'Mike, Michael, and Pete': 'Mike, Michael, Pete',
    'The Boys (except Cartman) and Dr. Phillips' : 'Stan, Kyle, Kenny, Dr. Phillips'
}

commas_conflictive = ['You And Me, Girl', 'The New Lyrics, part II', 'Stuart and wife, and Receptionist',
                     'Sixth Graders 2/3','A singer, Goth Kids (singing)']


for character in commas_conflictive:
    df = df[df['Character']!=character]

for i in range(len(df)):
    line = df.iloc[i,:]
    character = line['Character']
    if character in df_commas:
        if character in dict_commas_conflictive.keys():
            character = dict_commas_conflictive[character]
            df.iloc[i,:]['Character'] = character
        if ',' in character:
            character = character.split(',')
        elif '/' in character:
            character = character.split('/')
        elif ' and ' in character:
            character = character.split(' and ')
            
        for character_element in character:
            line['Character'] = character_element
            df = df.append(line, ignore_index=True)


df_commas_aux = [character for character in df_commas if character not in commas_conflictive]
            
for character_non_valid in df_commas_aux:
    df = df[df['Character']!=character_non_valid]
    
df = df.sort_values(by=['Season', 'Episode'], ignore_index=True)

#  1.4. Personajes con varios nombres asociados 

Hay personajes que aparecen con distinto nombre a lo largo de la serie, ya sea por errata, cambio de mayúsculas, minúsculas, puntos, etc. Quitamos todas las versiones posible de cada personaje.

In [ ]:
#for character in df['Character'].unique():
#    ratio = difflib.SequenceMatcher(None, 'Cartman', character).ratio()
#    if ratio>0.6:
#        print(character + ': ' + str(ratio))

# 1.5. Personajes con menos de diez líneas de diálogo
# 
Identificamos los personajes con menos de diez líneas de diálogo, puesto que puede haber cuestiones dentro del análisis para las que no nos interese considerarlos.

In [ ]:
for character in df['Character'].unique()[:50]:
    len_character = len(df[df['Character']==character])
    if len_character <= 10:
        print(character)
        
## Por cuestiones de compilación, el código ahora mismo solo muestran los personajes con menos
## de diez líneas entre los cincuenta primeros, en orden de aparición. Se cambiará.

## 2. Generamos Word Clouds

Generamos una nube de palabras con las palabras más dichas por los personajes principales a lo largo de toda la serie

In [ ]:
df['Line'] = [a.lower() for a in df['Line']]

In [ ]:
def generate_wordcloud(words, mask, colors):
    #word_cloud = WordCloud(width = 1600, height = 1600, background_color= None, mode='RGBA',color_func = colors, max_font_size= 400, stopwords=STOPWORDS, mask=mask).generate(words)
    word_cloud = WordCloud(width = 1600, height = 1600, background_color= None, mode='RGBA',color_func = colors, max_font_size= 400, stopwords=STOPWORDS, mask=mask).generate(words)
    word_cloud.generate(words)

    plt.figure(figsize = (11, 11))
    plt.imshow(word_cloud)
    plt.axis('off')

In [ ]:
text_cartman = ''

df_cartman = df[df['Character']=='Cartman']

for i in range(len(df_cartman)):
    line = df_cartman.iloc[i,:]
    text_cartman += ' ' + line['Line'] + ' '
    
text_cartman = text_cartman.replace('\n', ' ')

#mask_cartman = np.array(Image.open('../input/pictures/cartman.png'))
mask_cartman = imageio.imread('../input/pictures/cartman.png', pilmode='RGB')
colors_cartman = ImageColorGenerator(mask_cartman)


#INFO: https://amueller.github.io/word_cloud/generated/wordcloud.WordCloud.html
#RECURSOS IMÁGENES: https://www.freepng.es/

In [ ]:
#print(text_cartman[:100])

generate_wordcloud(text_cartman, mask_cartman,colors_cartman) # Arreglar

## HAY QUE MEJORARLO, USAREMOS:  https://www.datacamp.com/community/tutorials/wordcloud-python

In [ ]:
text_stan = ''

df_stan = df[df['Character']=='Stan']

for i in range(len(df_stan)):
    line = df_stan.iloc[i,:]
    text_stan += ' ' + line['Line'] + ' '
    
text_stan = text_stan.replace('\n', ' ')

#mask_stan = np.array(Image.open('../input/pictures/stan.png'))
mask_stan = imageio.imread('../input/pictures/stan.png', pilmode='RGB')
colors_stan = ImageColorGenerator(mask_stan)

In [ ]:
generate_wordcloud(text_stan, mask_stan, colors_stan)

In [ ]:
text_kenny = ''

df_kenny = df[df['Character']=='Kenny']

for i in range(len(df_kenny)):
    line = df_kenny.iloc[i,:]
    text_kenny += ' ' + line['Line'] + ' '
    
text_kenny = text_kenny.replace('\n', ' ')

mask_kenny = np.array(Image.open('../input/pictures/kenny.png'))
colors_kenny = ImageColorGenerator(mask_kenny)

In [ ]:
generate_wordcloud(text_kenny, mask_kenny, colors_kenny) # Arreglar

In [ ]:
#Falta limpiar paréntesis

text_kyle = ''

df_kyle = df[df['Character']=='Kyle']

for i in range(len(df_kyle)):
    line = df_kyle.iloc[i,:]
    text_kyle += ' ' + line['Line'] + ' '
    
text_kyle = text_kyle.replace('\n', ' ')

mask_kyle = imageio.imread('../input/pictures/kyle.png', pilmode='RGB')
colors_kyle = ImageColorGenerator(mask_kyle)

In [ ]:
generate_wordcloud(text_kyle, mask_kyle, colors_kyle)

In [ ]:
text_towelie = ''

df_towelie = df[df['Character']=='Towelie']

for i in range(len(df_towelie)):
    line = df_towelie.iloc[i,:]
    text_towelie += ' ' + line['Line'] + ' '
    
text_towelie = text_towelie.replace('\n', ' ')

mask_towelie = np.array(Image.open('../input/pictures/towelie.png'))
colors_towelie = ImageColorGenerator(mask_towelie)

In [ ]:
generate_wordcloud(text_towelie, mask_towelie, colors_towelie)

In [ ]:
text_butters = ''

df_butters = df[df['Character']=='Butters']

for i in range(len(df_butters)):
    line = df_butters.iloc[i,:]
    text_butters += ' ' + line['Line'] + ' '
    
text_butters = text_butters.replace('\n', ' ')

mask_butters = np.array(Image.open('../input/pictures/butters.png'))
colors_butters = ImageColorGenerator(mask_butters)

In [ ]:
generate_wordcloud(text_butters, mask_butters, colors_butters)

# 3. Ver qué personajes aparecen en todas las temporadas

In [ ]:
allseasons = df['Season'].unique()
characters_in_all_seasons = df['Character'].unique()
# Ya tenemos todos los personajes y todas las temporadas.
print("Total number of unique characters: {}".format(len(characters_in_all_seasons)))
# Comparamos ahora para cada temporada, dentro de la columna 'Season' del df, aquellos que se encuentren en la lista que hemos creado
# con los personajes. Estamos buscando caractéres dentro de una lista que hemos creado y que tiene formato string.

In [ ]:
for season in allseasons:
    character = df[df['Season'] == season].Character.unique()
    characters_in_all_seasons = [name for name in characters_in_all_seasons if name in character]
print("Number of characters who appear in all 18 seasons: {}".format(len(characters_in_all_seasons)))
print(characters_in_all_seasons)

# 4. Análisis de sentimientos

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()

# https://www.nltk.org/api/nltk.sentiment.html

In [ ]:
print('Cartman: ' + str(vader.polarity_scores(text_cartman)))
print('Kyle: ' + str(vader.polarity_scores(text_kyle)))
print('Stan: ' + str(vader.polarity_scores(text_stan)))
print('Kenny: ' + ('Polarity:' +str(vader.polarity_scores(text_kenny))+','+'Polarity:' +str(vader.polarity_scores(text_kenny))+','+'Polarity:' +str(vader.polarity_scores(text_kenny)))

In [ ]:
print(TextBlob(text_cartman).sentiment)
print(TextBlob(text_kyle).sentiment)
print(TextBlob(text_stan).sentiment)
print(TextBlob(text_kenny).sentiment)

In [ ]:
episode1 = df[(df['Season']==1) & (df['Episode']==1)]

In [ ]:
text_episode1 = ''

for line in episode1['Line']:
    text_episode1 += ' ' + line

In [ ]:
print('Episode 1: ' + str(vader.polarity_scores(text_episode1)))

print('Episode 1: ' + str(TextBlob(text_episode1).sentiment))

In [ ]:
for line in df['Line']:    
    words = line.split(' ')
    for word in words:
        ratio = difflib.SequenceMatcher(None, 'fuck', word).ratio()
        if ratio>0.7:
            print(word + ': ' + str(ratio))

In [ ]:
#from nltk.sentiment.vader import VaderConstants


#from nltk.sentiment.vader import SentimentIntensityAnalyzer
#vader = SentimentIntensityAnalyzer()



